In [0]:
!pip install mlflow

In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import RMSprop,Adam

import pickle

import mlflow

In [0]:
def preprocess(X_):

  X = X_.copy()

  ## Load_data
  preprocess_dict = pickle.load(open("/dbfs/FileStore/shared_uploads/agusbaffo@gmail.com/preprocess.pickle","rb"))
  sc = preprocess_dict["sc"]
  pca1 = preprocess_dict["pca1"]
  pca2 = preprocess_dict["pca2"]
  
  numerical_cols = ['Age',
    'Flight Distance',
    'Inflight wifi service',
    'Departure/Arrival time convenient',
    'Ease of Online booking',
    'Gate location',
    'Food and drink',
    'Online boarding',
    'Seat comfort',
    'Inflight entertainment',
    'On-board service',
    'Leg room service',
    'Baggage handling',
    'Checkin service',
    'Inflight service',
    'Cleanliness',
    'Departure Delay in Minutes',
    'Arrival Delay in Minutes']

  # Drop NaN
  X.dropna(axis=0, inplace=True)

  #Scaling the data
  X[numerical_cols] = sc.transform(X[numerical_cols])

  #Using PCA to reduce the dimensions of highly correlated features
  X['PCA1'] = pca1.transform(X[['Inflight wifi service', 'Ease of Online booking']])
  X['PCA2'] = pca2.transform(X[['Cleanliness', 'Inflight entertainment','Seat comfort','Food and drink']])

  # Drop columns
  X.drop(['Cleanliness','Inflight entertainment','Seat comfort','Food and drink','Inflight wifi service',
                'Ease of Online booking','Gender','Unnamed: 0','id'], axis=1, inplace=True)
                
  #Mapping the Customer Type, Type of Travel and Class Columns in the Testing Data
  X['Customer Type'] = X['Customer Type'].map({'disloyal Customer': 0, 'Loyal Customer' :1})
  X['Type of Travel'] = X['Type of Travel'].map({'Personal Travel': 0, 'Business travel' :1})
  X['Class'] = X['Class'].map({'Eco': 0, 'Eco Plus' :1, 'Business': 2})

  return X

In [0]:
new_data = pd.read_csv("/dbfs/FileStore/shared_uploads/agusbaffo@gmail.com/new_data.csv")
prepro_new_data = preprocess(new_data)

/databricks/python/lib/python3.7/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator StandardScaler from version 1.0.2 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
 UserWarning)
/databricks/python/lib/python3.7/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator PCA from version 1.0.2 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
 UserWarning)

### Load model and predict new data

In [0]:
model_prod = mlflow.sklearn.load_model(model_uri=f"models:/aps_model/Production")
# model_prod

Out[11]: GridSearchCV(cv=None, error_score=nan,
 estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
 class_weight=None,
 criterion='entropy',
 max_depth=None,
 max_features='auto',
 max_leaf_nodes=None,
 max_samples=None,
 min_impurity_decrease=0.0,
 min_impurity_split=None,
 min_samples_leaf=1,
 min_samples_split=2,
 min_weight_fraction_leaf=0.0,
 n_estimators=100, n_jobs=None,
 oob_score=False,
 random_state=None, verbose=0,
 warm_start=False),
 iid='deprecated', n_jobs=None, param_grid={},
 pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
 scoring='f1', verbose=0)

In [0]:
model_stage = mlflow.sklearn.load_model(model_uri=f"models:/aps_model/Staging")
# model_stage

Out[12]: <keras.engine.sequential.Sequential at 0x7f0f0d520110>

In [0]:
preduct_pred = model_prod.predict(pd.DataFrame(prepro_new_data))
stage_pred = model_stage.predict(pd.DataFrame(prepro_new_data))